In [1]:
import xgboost as xgb
#Load Data Prepr Librarier
import pandas as pd
import numpy as np
#Load Plotly
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
from sklearn.model_selection import KFold
import statsmodels.formula.api as smf

In [9]:
#Import Data


#Get the data
datafile = pd.read_csv("All_data.csv").fillna(0)

datafile['Holiday'] = np.where(datafile['Week']==6,1,
                                           np.where(datafile['Week']==58,1,
                                                    np.where(datafile['Week']==110,1,
                                                             np.where(datafile['Week']==33,1,
                                                                      np.where(datafile['Week']==85,1,
                                                                               np.where(datafile['Week']==137,1,0))))))
#Add last discount

In [11]:
#Uplift factors
model_list = ["BASE2_ACTUAL_PRICE_y","BASE2_ACTUAL_PRICE_PROM", "ACTUAL_PRICE","DISCOUNT2_ACTUAL_PRICE_x", "BASE2_ACTUAL_PRICE_HP_C", "BASE2_ACTUAL_PRICE_HP_T", "DISCOUNT2_ACTUAL_PRICE_y","LAST_DISCOUNT"]
model_list = model_list +["VOLUME_LAG1", "VOLUME_LAG2", "VOLUME_LAG3" ,"VOLUME_AVGLAG2", "VOLUME_AVGLAG3"]

#Promotion effectivness
model_list = model_list +["Week from promotion start"] +['Week ' + str(p) for p in range(1, 8)]
model_list = model_list +["time_from_prev2"]+["time_from_prev_"+p for p in ["A","B","C","D","E"]]
model_list = model_list +['Promotion ' + p + ' dummy' for p in ["A", "B", "C", "D", "E"]]
model_list = model_list +['VOLUMES_LAST_PROM', 'TIME_LAST_PROM', 'AVG_VOL_LAST_PROM']


#Canibalization
model_list = model_list +["Price_Diff_" + str(p) for p in range(1,7)]+ ["Price_log_Diff_" + str(p) for p in range(1,7)]
model_list = model_list +["TYPE_COMPETITOR" + str(p) + "_PRICE_3" for p in range(1,8)] + ["BASE2_COMPETITOR" + str(p) +"_PRICE" for p in range(1,8)] + ["DISCOUNT2_COMPETITOR" + str(p) + "_PRICE" for p in range(1,8)] +["BASE1_COMPETITOR" + str(p) +"_PRICE" for p in range(1,8)]+["COMPETITOR" + str(p) +"_PRICE" for p in range(1,8)] + ["DISCOUNT1_COMPETITOR" + str(p) + "_PRICE" for p in range(1,8)] 


print(len(model_list))
for i in model_list:
    print(i)
X_columns = datafile.loc[:,model_list].columns
x_train = datafile.loc[:,model_list].values
y_train = datafile["VOLUME_OF_SALES"].values

89
BASE2_ACTUAL_PRICE_y
BASE2_ACTUAL_PRICE_PROM
ACTUAL_PRICE
DISCOUNT2_ACTUAL_PRICE_x
BASE2_ACTUAL_PRICE_HP_C
BASE2_ACTUAL_PRICE_HP_T
DISCOUNT2_ACTUAL_PRICE_y
LAST_DISCOUNT
VOLUME_LAG1
VOLUME_LAG2
VOLUME_LAG3
VOLUME_AVGLAG2
VOLUME_AVGLAG3
Week from promotion start
Week 1
Week 2
Week 3
Week 4
Week 5
Week 6
Week 7
time_from_prev2
time_from_prev_A
time_from_prev_B
time_from_prev_C
time_from_prev_D
time_from_prev_E
Promotion A dummy
Promotion B dummy
Promotion C dummy
Promotion D dummy
Promotion E dummy
VOLUMES_LAST_PROM
TIME_LAST_PROM
AVG_VOL_LAST_PROM
Price_Diff_1
Price_Diff_2
Price_Diff_3
Price_Diff_4
Price_Diff_5
Price_Diff_6
Price_log_Diff_1
Price_log_Diff_2
Price_log_Diff_3
Price_log_Diff_4
Price_log_Diff_5
Price_log_Diff_6
TYPE_COMPETITOR1_PRICE_3
TYPE_COMPETITOR2_PRICE_3
TYPE_COMPETITOR3_PRICE_3
TYPE_COMPETITOR4_PRICE_3
TYPE_COMPETITOR5_PRICE_3
TYPE_COMPETITOR6_PRICE_3
TYPE_COMPETITOR7_PRICE_3
BASE2_COMPETITOR1_PRICE
BASE2_COMPETITOR2_PRICE
BASE2_COMPETITOR3_PRICE
BASE2_COMPETITOR4

In [12]:
results = pd.DataFrame()
#row = pd.DataFrame({"Sample" : k, "0" : 0, "1" : 0, "2" : 0, "3" : 0, "4" : 0, "5" : 0, "6" : 0, "7" : 0, "8" : 0, "9" : 0}, index = [k],)

for i in range(1):
    print("ITERATION " + str(i))
    #XGB Model
    xgb_params = {}
    xgb_params['booster'] = ['gbtree','gblinear','dart'][2]
    xgb_params['objective'] = ['reg:linear','reg:gamma','reg:tweedie'][0]
    xgb_params['learning_rate'] = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1][1]
    xgb_params['eval_metric'] = ['rmse','mae','logloss','error','merror','mlogloss','ndcg','map'][0]
    xgb_params['colsample_bytree'] = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0][8]
    xgb_params['max_depth'] = [1, 2, 4, 6, 8, 10, 12, 14, 16, 20][3]
    xgb_params['subsample'] = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0][5]
    xgb_params['min_child_weight'] = [1, 2, 4, 5, 6, 8, 10, 20, 50, 100][2]
    xgb_params['n_estimators'] = [10,50,100,300,500,  800, 1000, 1500, 2000][2]
    num_boost_round = [5,10,25,50,100,200,250,300,500,900][3]
    
    NFOLDS = 5
    kf = KFold(n_splits = NFOLDS, random_state = 999)
    
    ntrain = x_train.shape[0]
    
    oof_train = np.zeros((ntrain,))
    oof_test_skf = np.empty((NFOLDS, ntrain))
    
    #Go through each FOLD, solve the problem and store the results for the train and test samples
    k = 0
    temp1 = 0
    temp2 = 0
    for train_index, test_index in kf.split(x_train):
        x_tr = x_train[train_index]
        y_tr = y_train[train_index]
        x_te = x_train[test_index]
        y_te = y_train[test_index]
    
        dtrain = xgb.DMatrix(x_tr, y_tr, feature_names = X_columns)
        dtrain_test = xgb.DMatrix(x_te, feature_names = X_columns)
        gbm = xgb.train(dict(xgb_params, silent = 1), dtrain, num_boost_round = num_boost_round)
    
        #Check the results
        temp1 += np.power(pd.concat([pd.DataFrame(y_tr), pd.DataFrame(pd.DataFrame(gbm.predict(dtrain).tolist()).iloc[:,0])], axis = 1).corr().iloc[1,0],2)
        temp2 += np.power(pd.concat([pd.DataFrame(y_te), pd.DataFrame(pd.DataFrame(gbm.predict(dtrain_test).tolist()).iloc[:,0])], axis = 1).corr().iloc[1,0],2)
        k = k + 1
    print(temp1/k)
    print(temp2/k)


#table_macro = pd.concat([table_macro, row], axis = 0)

#corr_app_sale = pd.concat([t_predict_s2, t_predict_sale, t_predict_build ,t_predict_macro], axis = 1).corr()


ITERATION 0
0.9928868063285436
0.6440422247688409


In [13]:
fimp = pd.Series(gbm.get_score(), name = "Score").sort_values(ascending = False)
print(fimp)

Price_Diff_3                   34
BASE2_ACTUAL_PRICE_HP_C        26
VOLUME_LAG3                    26
VOLUME_LAG2                    24
ACTUAL_PRICE                   23
VOLUME_LAG1                    23
VOLUME_AVGLAG2                 21
Price_log_Diff_1               19
BASE2_ACTUAL_PRICE_y           18
BASE2_ACTUAL_PRICE_PROM        17
COMPETITOR1_PRICE              16
BASE2_ACTUAL_PRICE_HP_T        16
DISCOUNT2_COMPETITOR1_PRICE    15
BASE2_COMPETITOR1_PRICE        13
Price_Diff_1                   13
VOLUMES_LAST_PROM              13
time_from_prev_A               13
time_from_prev_B               12
VOLUME_AVGLAG3                 11
DISCOUNT1_COMPETITOR1_PRICE    11
AVG_VOL_LAST_PROM              11
time_from_prev_C               10
BASE1_COMPETITOR1_PRICE         9
LAST_DISCOUNT                   9
Price_Diff_4                    7
DISCOUNT2_ACTUAL_PRICE_x        7
time_from_prev_D                6
BASE2_COMPETITOR5_PRICE         5
Price_log_Diff_3                5
Price_Diff_5  

In [14]:
#Copy File
datafile_2 = datafile.copy()

#Define list of no-log
nolog_list = ["Week from promotion start"] +['Week ' + str(p) for p in range(1, 8)] + ["TIME_LAST_PROM"]
nolog_list = nolog_list +["time_from_prev2"]+["time_from_prev_"+p for p in ["A","B","C","D","E"]]
nolog_list = nolog_list +['Promotion ' + p + ' dummy' for p in ["A", "B", "C", "D", "E"]]
nolog_list = nolog_list +["TYPE_COMPETITOR" + str(p) + "_PRICE_3" for p in range(1,8)]
nolog_list = nolog_list +["Price_log_Diff_" + str(p) for p in range(1,7)]

formula = "datafile_2['VOLUME_OF_SALES'] ~ "
#log_list = list(fimp.index)

log_list = []
for i in model_list:
    if i not in ['ACTUAL_PRICE'] + nolog_list:
        log_list = log_list + [i]

k = 0
for i in log_list:
    if k == 0:
        formula = formula + " datafile_2['" + i + "']"
    else:
        formula = formula + " + datafile_2['" + i + "']"
    
    k = k + 1
    #if i not in nolog_list:
    #    datafile_2[i] = np.log(datafile_2[i].replace(0, np.nan))
    #    datafile_2[i] = datafile_2[i].replace(np.nan, 0)

#Log volume
#datafile_2["VOLUME_OF_SALES"] = np.log(datafile_2["VOLUME_OF_SALES"])       
        
model1 = smf.ols(formula=formula, data=datafile_2).fit()
print(model1.summary())

                                  OLS Regression Results                                 
Dep. Variable:     datafile_2['VOLUME_OF_SALES']   R-squared:                       0.856
Model:                                       OLS   Adj. R-squared:                  0.783
Method:                            Least Squares   F-statistic:                     11.66
Date:                           Thu, 26 Dec 2019   Prob (F-statistic):           4.99e-24
Time:                                   20:37:43   Log-Likelihood:                -1625.0
No. Observations:                            146   AIC:                             3350.
Df Residuals:                                 96   BIC:                             3499.
Df Model:                                     49                                         
Covariance Type:                       nonrobust                                         
                                                coef    std err          t      P>|t|      [0.025   

In [15]:
formula = "datafile['VOLUME_OF_SALES'] ~ "
#for i in list(fimp.index):
#    formula = formula + " + datafile['" + i + "']"
remove_list = ['BASE2_COMPETITOR6_PRICE','Price_log_Diff_5','DISCOUNT2_COMPETITOR5_PRICE','DISCOUNT2_COMPETITOR5_PRICE','BASE2_COMPETITOR1_PRICE','DISCOUNT2_COMPETITOR4_PRICE','VOLUMES_LAST_PROM','Price_log_Diff_4','Price_log_Diff_1','BASE2_COMPETITOR3_PRICE','DISCOUNT2_COMPETITOR3_PRICE','BASE2_COMPETITOR7_PRICE','BASE2_COMPETITOR5_PRICE','DISCOUNT2_COMPETITOR2_PRICE','BASE2_COMPETITOR4_PRICE','DISCOUNT2_COMPETITOR1_PRICE','DISCOUNT2_COMPETITOR6_PRICE','DISCOUNT2_COMPETITOR7_PRICE','BASE2_COMPETITOR2_PRICE','VOLUME_AVGLAG2','VOLUME_LAG2','BASE2_ACTUAL_PRICE_HP_T','Price_Diff_6','COMPETITOR5_PRICE','COMPETITOR7_PRICE','time_from_prev_A','time_from_prev_C','time_from_prev_D','AVG_VOL_LAST_PROM','TYPE_COMPETITOR5_PRICE_3','TYPE_COMPETITOR1_PRICE_3','BASE1_COMPETITOR6_PRICE','BASE1_COMPETITOR7_PRICE','Week 3','Price_Diff_2','Price_Diff_6''Week 3','BASE1_COMPETITOR3_PRICE','DISCOUNT1_COMPETITOR7_PRICE','DISCOUNT1_COMPETITOR3_PRICE','DISCOUNT1_COMPETITOR2_PRICE','Price_Diff_1','DISCOUNT1_COMPETITOR4_PRICE','Price_Diff_3','BASE2_ACTUAL_PRICE_PROM','COMPETITOR4_PRICE','COMPETITOR6_PRICE','Week 7','time_from_prev_B','TYPE_COMPETITOR7_PRICE_3','VOLUME_LAG3','Promotion E dummy','VOLUME_AVGLAG3','BASE2_ACTUAL_PRICE_HP_C','BASE1_COMPETITOR4_PRICE','Price_log_Diff_2','Week from promotion start','TYPE_COMPETITOR3_PRICE_3','Price_Diff_5','COMPETITOR3_PRICE','COMPETITOR2_PRICE','ACTUAL_PRICE','Promotion B dummy','TYPE_COMPETITOR6_PRICE_3','DISCOUNT1_COMPETITOR5_PRICE','TYPE_COMPETITOR4_PRICE_3','TYPE_COMPETITOR6_PRICE_3''DISCOUNT1_COMPETITOR5_PRICE','BASE1_COMPETITOR5_PRICE','Week 4','Price_Diff_4','Week 2','BASE1_COMPETITOR2_PRICE','BASE1_COMPETITOR1_PRICE','DISCOUNT1_COMPETITOR1_PRICE', 'Week 6','DISCOUNT2_ACTUAL_PRICE_x','COMPETITOR1_PRICE','Week 5','Price_log_Diff_6','DISCOUNT1_COMPETITOR6_PRICE', 'Competitor3_Price_Impact','Promotion C dummy', 'Competitor1_Price_Impact', 'TYPE_COMPETITOR2_PRICE_3', 'Promotion D dummy','time_from_prev_E','time_from_prev2']

k = 0
corr_list = []
for i in model_list:
    if i not in remove_list:
        corr_list = corr_list + [i]
        if k == 0:
            formula = formula + " datafile['" + i + "']"
        else:
            formula = formula + " + datafile['" + i + "']"

        k = k + 1
corr_list = corr_list + ["VOLUME_OF_SALES"]

model = smf.ols(formula=formula, data=datafile).fit()
print(model.summary())

                                 OLS Regression Results                                
Dep. Variable:     datafile['VOLUME_OF_SALES']   R-squared:                       0.796
Model:                                     OLS   Adj. R-squared:                  0.784
Method:                          Least Squares   F-statistic:                     66.63
Date:                         Thu, 26 Dec 2019   Prob (F-statistic):           1.78e-43
Time:                                 20:37:50   Log-Likelihood:                -1650.6
No. Observations:                          146   AIC:                             3319.
Df Residuals:                              137   BIC:                             3346.
Df Model:                                    8                                         
Covariance Type:                     nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
---------------

In [18]:
for i in range(2,3):
    formula = "datafile['VOLUME_OF_SALES'] ~ "
    model_list2 = ['BASE2_ACTUAL_PRICE_y','DISCOUNT2_ACTUAL_PRICE_y','LAST_DISCOUNT','VOLUME_LAG1','Week 1','Promotion A dummy','TIME_LAST_PROM']
    model_list2 = model_list2 + ["Price_Relative_Diff__3"]
    model_list2 = model_list2 + ["COMP_1_only", "Holiday"]

    
    k = 0
    corr_list = []
    for i in model_list2:
        if k == 0:
            formula = formula + " datafile['" + i + "']"
        else:
            formula = formula + " + datafile['" + i + "']"

        k = k + 1
    corr_list = corr_list + ["VOLUME_OF_SALES"]

    model = smf.ols(formula=formula, data=datafile).fit()
    print(model.summary())


                                 OLS Regression Results                                
Dep. Variable:     datafile['VOLUME_OF_SALES']   R-squared:                       0.810
Model:                                     OLS   Adj. R-squared:                  0.796
Method:                          Least Squares   F-statistic:                     57.45
Date:                         Thu, 26 Dec 2019   Prob (F-statistic):           9.92e-44
Time:                                 20:39:46   Log-Likelihood:                -1645.4
No. Observations:                          146   AIC:                             3313.
Df Residuals:                              135   BIC:                             3346.
Df Model:                                   10                                         
Covariance Type:                     nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
---------------